# Cirq basic


we will look at the basics of how to use Cirq. I will talk about how to use qubits, different devices, gates, operations, unitary matrices, adding noise and run a couple of circuits


installing Cirq

In [ ]:
!pip install --quiet cirq
!pip install cirq-google

import cirq

import cirq_google

## Qubits

Cirq has 3 ways to initialise qubits

  .NamedQubits: here we can give names to the qubits. \
  .LineQubits: Here we can create qubits using arrays. \
  .GridQubits: Here qubits are a part of a grid which we can manipulate ([source](https://quantumai.google/reference/python/cirq/GridQubit)).


Here are some examples of defining each type of qubit.

In [ ]:
# named qubits (primarily used when we want to give names to qubits for algorithms)
q0 = cirq.NamedQubit('alice')
q1 = cirq.NamedQubit('bob')


# we can create individual qubits that are part of a lined arch.
q2 = cirq.LineQubit(2)

# we can even use the python Range function to create multiple at once
q0, q1, q2 = cirq.LineQubit.range(3)

# We can create qubits that are a part of a grid (more useful when working on actual hardware).
q4_5 = cirq.GridQubit(4, 5)

# Or using shapes like square, rectangle. (you can also use custom shapes)
qubits = cirq.GridQubit.rect(rows = 2, cols = 4 )
print(qubits)

There are also pre-packaged sets of qubits called [Devices](https://quantumai.google/cirq/hardware/devices).  These follow the architecture of famous hardware that can be imported. Cirq also lets us make api calls to popular hardware such as AQT, Azure etc (check devices documantation) and use their mappings.

In [ ]:
#this is the diamond-shaped grid with 54 qubits that mimics the sycamore device from google.

print(cirq_google.Sycamore)

## Gates and operations

Now we can use the qubits to create operations that can be used to create circuits [Documentation](https://quantumai.google/cirq/build/gates).

There are single gate operations such as Hadamard, Pauli X Y and Z gates and a lot more which are applied to single gates. \


There's also Multi gate operations such as CNOT, SWAP, CZ. \


There's a Measurement operation as well which measures the qubit. \


examples:

In [ ]:
Hadamard = cirq.H
PauliZ = cirq.Z
SWAPGate = cirq.SWAP

#we can even create roots of gates such as X  to get sqrt(X) these can be used for more granular rotations compared to PauliX
SqrtPauliX = cirq.X**0.5

# Example operations
q0,q1 = cirq.LineQubit.range(2)
print("q0, q1:",q0,q1)
z_op = cirq.Z(q0)
print(z_op)
not_op = cirq.CNOT(q0, q1)
print(not_op)
sqrt_iswap_op = cirq.SQRT_ISWAP(q0, q1)

# You can also use the gates you specified earlier.
swap = SWAPGate(q0, q1)
pauliZ = PauliZ(q0)
print(swap)

cirq also lets us create custom gates. You can find more [here](https://quantumai.google/cirq/build/custom_gates)

## Circuits and moments

To see how these gates work in a circuit we will now build circuits. Cirq also has this concept of moments. These can be thought of as time slices that seperate when each gate is executed in a circuit (better explained via example)(Think of these as times t1, t2 etc where we are interested in whats happening in the circuit or want to control which gates occur first.). \

Cirq explains a Quantum "Circuit" as a collection of the aforementioned "Moments" and moments themselves as a collections of gate operations on the qubits. \

 A `Moment` is a collection of `Operation`s that all act during the same time slice. A `Moment` can be thought of as a vertical slice of a quantum circuit diagram. \

Cirq is optimised and by default will attempt to slide your operation into the earliest possible `Moment` when you insert it so if you want something different then we must pay attention to where we are placing moments.

Circuits doccumentation in detail [here](https://quantumai.google/reference/python/cirq/Circuit)

In [ ]:
#creating a simple circuit with 3 qubits and applyinh a Hadamard to each of them
circuit = cirq.Circuit()
qubits = cirq.LineQubit.range(3)
circuit.append(cirq.H(qubits[0]))
circuit.append(cirq.H(qubits[1]))
circuit.append(cirq.H(qubits[2]))
print(circuit)

In [ ]:
#using python list comprehention to make it easier
circuit = cirq.Circuit()
ops = [cirq.H(q) for q in cirq.LineQubit.range(3)]
circuit.append(ops)
print(circuit)

In [ ]:
circuit = cirq.Circuit()
circuit.append([cirq.SWAP(q0, q1), cirq.H(q2)])
print(circuit)

All the Hadamards are made to occure at the same time in the circuits 1 and 2 above. in circuit 3 Hadamard Op on q2 is pushed ot the left. Cirq pushes non overlapping operations to the left by default. For overlaping gates they are put one after the other like in circuit 3.

In [ ]:
circuit3 = cirq.Circuit()
circuit3.append([cirq.SWAP(q0, q1), cirq.H(q1),  cirq.H(q2),cirq.CNOT(q1,q2)])
print(circuit3)

What if we want the hadamards in circuit 3 to occur together (no reason). This is where the "Moments"" are usefull.

you can create the circuit moment-by-moment or use a different `InsertStrategy`, explained clearly [Here](https://quantumai.google/cirq/build/circuits).

basically we add time slices to our circuits and organise our circuit using them.



In [ ]:
# Creates each gate in a separate moment by passing an iterable of Moments instead of Operations.
print(cirq.Circuit(cirq.Moment([cirq.H(q)]) for q in cirq.LineQubit.range(3)))

In [ ]:
# for circuit 3 (Moving both hadamard together)
from cirq.circuits import InsertStrategy

circuit32 = cirq.Circuit()
moment = cirq.Moment(cirq.H(q1),  cirq.H(q2))
circuit32.append([cirq.SWAP(q0, q1), moment,cirq.CNOT(q1,q2)])
print(circuit32)

## Unitary matrices

Many quantum operations have unitary matrix representations.  This matrix can be accessed by applying `cirq.unitary(operation)` to that `operation`.  This can be applied to gates, operations, and circuits that support this protocol and will return the unitary matrix that represents the object.

In [ ]:
print('Unitary of the X gate')
print(cirq.unitary(cirq.X))

print('Unitary of SWAP operator on two qubits.')
q0, q1 = cirq.LineQubit.range(2)
print(cirq.unitary(cirq.SWAP(q0, q1)))

print('Unitary of a sample circuit')
print(cirq.unitary(cirq.Circuit(cirq.X(q0), cirq.SWAP(q0, q1))))

In [ ]:
#You can also convert Unitary to custom gates as shown in the link above:
import numpy as np
"""Define a custom single-qubit gate."""
class MyGate(cirq.Gate):
    def __init__(self):
        super(MyGate, self)

    def _num_qubits_(self):
        return 1

    def _unitary_(self):
        return np.array([
            [1.0,  1.0],
            [-1.0, 1.0]
        ]) / np.sqrt(2)

    def _circuit_diagram_info_(self, args):
        return "G"

my_gate = MyGate()
print(cirq.unitary(my_gate))

##Noise

Cirq allows us to add noise to circuits at multiple levels.

we can add noise to the entire circuit, while taking measurements (documentation of both these might be incomplete wasnt able to make it run) and add noise to intividual bits and connections.

We can see cirq implementation of a couple of types noise below.

[Full Documentation on Noise](https://quantumai.google/cirq/noise/representing_noise)

### Bit flip

`cirq.BitFlipChannel` (or `cirq.bit_flip`) is equivalent to applying `cirq.X` with a given probability. This channel is best used to represent state-agnostic bit flip errors in the body of a circuit.

In [ ]:
q0 = cirq.LineQubit(0)
circuit = cirq.Circuit(
    cirq.bit_flip(p=0.5).on(q0),
    cirq.measure(q0, key='result')
)
result = cirq.Simulator(seed=0).run(circuit, repetitions=1000)
print(result.histogram(key='result'))

### Amplitude damping

`cirq.AmplitudeDampingChannel` (or `cirq.amplitude_damp`) performs a $|1\rangle \rightarrow |0\rangle$ transformation with some probability `gamma`, leaving the existing $|0\rangle$ state alone. This channel is best used to represent an idealized form of energy dissipation, where qubits decay from $|1\rangle$ to $|0\rangle$.

In [ ]:
q0 = cirq.LineQubit(0)
circuit = cirq.Circuit(
    cirq.X(q0),
    cirq.amplitude_damp(gamma=0.2).on(q0),
    cirq.measure(q0, key='result')
)
result = cirq.Simulator(seed=0).run(circuit, repetitions=1000)
print(result.histogram(key='result'))

## Simulation

Once we have built our circuits with the necesary noise we can get the simulated results of running our circuit using cirq's "Simulator()" (20 qubit limit)

[Documentation](https://quantumai.google/cirq/simulate/simulation)

There are two different approaches to using a simulator:

`simulate()`:  Offers a detailed, comprehensive look at the quantum state (wavefunction) after circuit execution. It is for when you need to look "under the hood" of your quantum circuit. It's output typically includes probabilities, phase information, and can even show how entanglements and superpositions are created within the circuit.
`run()`: Outputs measurement results as bit strings, simulating the practical outcome of running a quantum circuit on a physical device.



In [ ]:
#simple hadamard
simple_hadamard = cirq.Circuit()
q0 = cirq.LineQubit(0)
simple_hadamard.append(cirq.H(q0))
s = cirq.Simulator()
results = s.simulate(simple_hadamard)
print(results)


In [ ]:
#adding one more hadamard (retirn to |0>)
simple_hadamard.append(cirq.H(q0))
results = s.simulate(simple_hadamard)
print(results)

We can take a look at the example simulation a 2-qubit "Bell State" shown in the documentation:

In [ ]:
# Create a circuit to generate a Bell State:
# 1/sqrt(2) * ( |00⟩ + |11⟩ )
bell_circuit = cirq.Circuit()
q0, q1 = cirq.LineQubit.range(2)
bell_circuit.append(cirq.H(q0))
bell_circuit.append(cirq.CNOT(q0, q1))

# Initialize Simulator
s = cirq.Simulator()

print('Simulate the circuit:')
results = s.simulate(bell_circuit)
print(results)

# For sampling, we need to add a measurement at the end
bell_circuit.append(cirq.measure(q0, q1, key='result'))

# Sample the circuit
sample = s.run(bell_circuit, repetitions=1000)


In [ ]:
print(sample)

## Visualizing Results

When you use `run()` to get a sample distribution of measurements, you can directly graph the simulated samples as a histogram with `cirq.plot_state_histogram`.

In [ ]:
import matplotlib.pyplot as plt

cirq.plot_state_histogram(sample, plt.subplot())
plt.show()

Here I've created the quantum teleportation circuit and simulated it using cirq.

[creating classical comm line](https://quantumai.google/cirq/build/classical_control)

In [ ]:
def quantum_teleportation_circuit(gate):
    circuit = cirq.Circuit()
    alice = cirq.NamedQubit('alice')
    msg = cirq.NamedQubit('msg')
    bob = cirq.NamedQubit('bob')

    #entangles qubit betweeen  Alice and Bob
    circuit.append([cirq.H(alice), cirq.CNOT(alice, bob)])


    # Creates a message to send.
    circuit.append(gate(msg))


    # Bell measurement of the Message and Alice's entangled qubit.
    circuit.append([cirq.CNOT(msg, alice), cirq.H(msg), cirq.measure(msg,alice)])


    # Uses the two classical bits from the Bell measurement to recover the
    # original quantum Message on Bob's entangled qubit.
    circuit.append([cirq.CNOT(alice, bob), cirq.CZ(msg,bob)])

    return circuit

In [ ]:
# gate for msg qubit Pauili X

gate = cirq.X

#create circuit
circuit = quantum_teleportation_circuit(gate)
print(circuit)

In [ ]:
#block sphere
import numpy as np
message = cirq.Circuit(gate.on(cirq.NamedQubit('msg'))).final_state_vector()
message_bloch_vector = cirq.bloch_vector_from_state_vector(message,index=0)

print("bloch sphere")
print(message_bloch_vector)

In [ ]:
# gate for msg qubit Pauili Z
gate = cirq.Z

#create circuit
circuit = quantum_teleportation_circuit(gate)
print(circuit)

In [ ]:
message = cirq.Circuit(gate.on(cirq.NamedQubit('msg'))).final_state_vector()
message_bloch_vector = cirq.bloch_vector_from_state_vector(message,index=0)

print("bloch sphere")
print(message_bloch_vector)

Tried Noisy circuit but got errors